In [7]:
# Data prep
import os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from random import random
import numpy as np
keras.mixed_precision.set_global_policy('mixed_float16')

# data loading config
batch_size = 64
img_height = 28
img_width = 28
dataPath = 'fashion_mnist'
labels = 'inferred'  # auto find from folders
label_mode = 'categorical'  # one hot encoding
color_mode = 'grayscale'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE


def split_train_test(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            if randomNum >= test_split:
                pass
            else:
                os.makedirs('test\\'+root+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+root+'\\')

# split_train_test('fashion_mnist')  # Only need to run this once

print('Training data')
train = keras.preprocessing.image_dataset_from_directory(dataPath, labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, subset="training", seed=seed, validation_split=test_split,
image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names

val = keras.preprocessing.image_dataset_from_directory(dataPath, labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, subset="validation", seed=seed, validation_split=test_split,
image_size=(img_height, img_width), batch_size=batch_size)

print('\nTesting data')
test = keras.preprocessing.image_dataset_from_directory('test\\fashion_mnist', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed,
image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

print()
print('train classes:', train_class_names)
print('test classes:', test_class_names)

Training data
Found 56075 files belonging to 10 classes.
Using 44860 files for training.
Found 56075 files belonging to 10 classes.
Using 11215 files for validation.

Testing data
Found 13925 files belonging to 10 classes.

train classes: ['Ankle Boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'T-Shirt', 'Trouser']
test classes: ['Ankle Boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'T-Shirt', 'Trouser']


In [8]:
# keras.Dense implementation
# model params
import tensorflow_transform as tft
import tensorflow_probability as tfp
opt = keras.optimizers.Adam(lr=1e-3, epsilon=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
epoch = 100
batch_size = 128
epsilon = 1e-7
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=10, verbose=1)
]

def gayYiqinNorm(x):
    return (x - np.mean(x))/tf.math.sqrt(np.var(x)+epsilon)

# model layers
xInput = layers.Input((28, 28))  # get resolutions, ignore batch size
x = layers.Flatten()(xInput)
x = layers.experimental.preprocessing.Rescaling(1./255)(x)  # fit 0-255 into 0 and 1
x = gayYiqinNorm(x)
x = layers.Dense(128)(x)
x = layers.Activation('relu')(x)
x = gayYiqinNorm(x)
x = layers.Dense(512)(x)
x = layers.Activation('relu')(x)
x = gayYiqinNorm(x)
x = layers.Dense(len(train_class_names))(x)  # a probability for each class so need same no as classes
xOutput = layers.Activation('softmax')(x)

model = keras.Model(xInput, xOutput)
model.compile(optimizer=opt, loss=loss, metrics='accuracy')
model.summary()
model.fit(train, validation_data=val, batch_size=batch_size, epochs=epoch, callbacks=callbacks, verbose=1)

RuntimeError: Failed to apply Operation <class 'tensorflow_transform.analyzer_nodes.TensorSource'>, with error: tensor must be a Tensor, got KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.float16, name=None), name='tf.cast_1/Cast:0', description="created by layer 'tf.cast_1'") of type <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>

In [3]:
model.evaluate(test)
model.save('Fashion MNIST')  # doesnt work with YiqinIsGay maybe Yiqin is too gay for TF to save

218/218 [==============================] - 5s 22ms/step - loss: 0.4127 - accuracy: 0.8995

In [4]:
# YiqinIsRacist implementation (saving doesnt work idk why)
class YiqinIsRacist(layers.Layer):
   def __init__(self, units):
      super(YiqinIsRacist, self).__init__()
      self.units = units
   def build(self, input_shape):
      self.Weight = self.add_weight(
         shape=(input_shape[-1], self.units),
         initializer="glorot_uniform",
         trainable=True,
      )
      self.Bias = self.add_weight(
         shape=(self.units,), initializer="random_normal", trainable=True
      )
   def call(self, inputs):
       print('Alistair is the gayest of all gays')
       return tf.matmul(inputs, self.Weight) + self.Bias

In [5]:
# mode params
opt = keras.optimizers.Adam(lr=1e-3, epsilon=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
epoch = 100
batch_size = 128
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    # tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),  # doesnt work with YiqinIsRacist maybe Yiqin is too racist for TF to save
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=10, verbose=1)
]

# model layers
xInput = layers.Input((28, 28))  # get resolutions, ignore batch size
x = layers.Flatten()(xInput)
x = layers.experimental.preprocessing.Rescaling(1./255)(x)  # fit 0-255 into 0 and 1
x = YiqinIsRacist(128)(x)
x = layers.Activation('relu')(x)
x = YiqinIsRacist(512)(x)
x = layers.Activation('relu')(x)
x = YiqinIsRacist(len(train_class_names))(x)  # a probability for each class so need same no as classes
xOutput = layers.Activation('softmax')(x)

model = keras.Model(xInput, xOutput)
model.compile(optimizer=opt, loss=loss, metrics='accuracy')
model.summary()
model.fit(train, validation_data=val, batch_size=batch_size, epochs=epoch, callbacks=callbacks, verbose=1)

Alistair is the gayest of all gays
Alistair is the gayest of all gays
Alistair is the gayest of all gays
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 784)               0         
_________________________________________________________________
yiqin_is_racist (YiqinIsRaci (None, 128)               100480    
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
_________________________________________________________________
yiqin_is_racist_1 (YiqinIsRa (None, 512)               66048     
____________________

In [6]:
model.evaluate(test)
# model.save('Fashion MNIST')  # doesnt work with YiqinIsRacist maybe Yiqin is too racist for TF to save

218/218 [==============================] - 0s 1ms/step - loss: 0.4523 - accuracy: 0.8964


[0.45233604311943054, 0.896373450756073]